In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import timm
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, Dataset

# Define the ResNet-ViT hybrid model with Dropout
class ResNetViT(nn.Module):
    def __init__(self):
        super(ResNetViT, self).__init__()
        # Load pre-trained ResNet50
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the classification layer

        # Load pre-trained Vision Transformer (ViT)
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()  # Remove the classification layer

        self.dropout = nn.Dropout(0.5)  # Dropout with a 50% rate

    def forward(self, x):
        x_resnet = self.resnet(x)
        x_vit = self.vit(x)
        x = torch.cat((x_resnet, x_vit), dim=1)
        x = self.dropout(x)  # Apply dropout
        return x

# Instantiate the model
model = ResNetViT()
model.eval()  # Set to evaluation mode

# Define image preprocessing with data augmentation
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomVerticalFlip(),    # Randomly flip images vertically
    transforms.RandomRotation(30),      # Randomly rotate images
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change the brightness, contrast, saturation and hue
    transforms.RandomGrayscale(p=0.2),  # Randomly convert images to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define custom dataset
class AppleDataset(Dataset):
    def __init__(self, image_paths, labels, preprocess):
        self.image_paths = image_paths
        self.labels = labels
        self.preprocess = preprocess

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.preprocess(img)
        label = self.labels[idx]
        return img, label

# Function to extract features
def extract_features(model, data_loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in tqdm(data_loader):
            features_batch = model(imgs)
            features.append(features_batch.numpy())
            labels.extend(lbls)
    return np.vstack(features), np.array(labels)

# Create dataset
def create_image_paths_dataset(paths: list, labels: list):
    image_paths = []
    image_labels = []
    for path, label in zip(paths, labels):
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        image_paths.extend([os.path.join(path, f) for f in files])
        image_labels.extend([label] * len(files))
    
    return pd.DataFrame({'image_path': image_paths, 'label': image_labels})

# Paths to datasets
scab_path = '/kaggle/input/plantvillage-dataset/color/Apple___Apple_scab/'
black_path = '/kaggle/input/plantvillage-dataset/color/Apple___Black_rot/'
rust_path = '/kaggle/input/plantvillage-dataset/color/Apple___Cedar_apple_rust/'
healthy_path = '/kaggle/input/plantvillage-dataset/color/Apple___healthy/'

paths = [scab_path, black_path, rust_path, healthy_path]
labels = ['scab', 'black_rot', 'cedar_rust', 'healthy']
apples = create_image_paths_dataset(paths, labels)

# Split the data into training, validation, and testing sets
image_paths = apples['image_path'].tolist()
labels = apples['label'].tolist()
X_train_paths, X_temp_paths, y_train, y_temp = train_test_split(image_paths, labels, test_size=0.3, random_state=42, stratify=labels)
X_val_paths, X_test_paths, y_val, y_test = train_test_split(X_temp_paths, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Create data loaders
train_dataset = AppleDataset(X_train_paths, y_train, preprocess)
val_dataset = AppleDataset(X_val_paths, y_val, preprocess)
test_dataset = AppleDataset(X_test_paths, y_test, preprocess)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Extract features from the training, validation, and testing sets
features_train, labels_train = extract_features(model, train_loader)
features_val, labels_val = extract_features(model, val_loader)
features_test, labels_test = extract_features(model, test_loader)

# Standardize the features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_val_scaled = scaler.transform(features_val)
features_test_scaled = scaler.transform(features_test)

# Apply PCA for feature selection
pca = PCA(n_components=10)  # Adjust the number of components as needed
features_train_reduced = pca.fit_transform(features_train_scaled)
features_val_reduced = pca.transform(features_val_scaled)
features_test_reduced = pca.transform(features_test_scaled)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels_train), y=labels_train)
class_weights_dict = {cls: weight for cls, weight in zip(np.unique(labels_train), class_weights)}

# Initialize logistic regression model with class weights
logistic_regression = LogisticRegression(max_iter=1000, penalty='l2', C=0.1, class_weight=class_weights_dict)

# Hyperparameter tuning using Grid Search
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')
grid_search.fit(features_train_reduced, labels_train)
best_model = grid_search.best_estimator_

# Evaluate on validation set
val_pred = best_model.predict(features_val_reduced)
val_accuracy = accuracy_score(labels_val, val_pred)
val_conf_matrix = confusion_matrix(labels_val, val_pred)
val_class_report = classification_report(labels_val, val_pred)

print(f"Validation Accuracy: {val_accuracy}")
print("Validation Confusion Matrix:")
print(val_conf_matrix)
print("Validation Classification Report:")
print(val_class_report)

# Make predictions on the test set using the best model from GridSearchCV
y_pred = best_model.predict(features_test_reduced)

# Evaluate the model on the test set
accuracy = accuracy_score(labels_test, y_pred)
conf_matrix = confusion_matrix(labels_test, y_pred)
class_report = classification_report(labels_test, y_pred, output_dict=True)
metrics = pd.DataFrame(class_report).transpose()

print(f"Test Accuracy: {accuracy}")
print("Test Confusion Matrix:")
print(conf_matrix)
print("Test Classification Report:")
print(metrics)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 15/15 [03:17<00:00, 13.19s/it]


Validation Accuracy: 0.8529411764705882
Validation Confusion Matrix:
[[ 85   1   4   3]
 [  2  35   1   3]
 [ 12   3 216  16]
 [ 10   8   7  70]]
Validation Classification Report:
              precision    recall  f1-score   support

   black_rot       0.78      0.91      0.84        93
  cedar_rust       0.74      0.85      0.80        41
     healthy       0.95      0.87      0.91       247
        scab       0.76      0.74      0.75        95

    accuracy                           0.85       476
   macro avg       0.81      0.84      0.82       476
weighted avg       0.86      0.85      0.85       476

Test Accuracy: 0.8760504201680672
Test Confusion Matrix:
[[ 82   1   6   4]
 [  1  38   0   3]
 [  7   0 218  22]
 [  5   4   6  79]]
Test Classification Report:
              precision    recall  f1-score    support
black_rot      0.863158  0.881720  0.872340   93.00000
cedar_rust     0.883721  0.904762  0.894118   42.00000
healthy        0.947826  0.882591  0.914046  247.00000
sca